In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from numpy.polynomial import legendre
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge

class OrthogonalPolynomialFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2, include_bias=True):
        self.degree = degree
        self.include_bias = include_bias

    def fit(self, X, y=None):
        X = np.asarray(X)
        self.X_min_ = X.min(axis=0)
        self.X_max_ = X.max(axis=0)
        n_features = X.shape[1]
        combos = []
        for comb in itertools.product(range(self.degree + 1), repeat=n_features):
            if sum(comb) <= self.degree and (self.include_bias or sum(comb) > 0):
                combos.append(comb)
        combos.sort(key=lambda c: (sum(c), c))
        self.combinations_ = combos
        return self

    def transform(self, X):
        X = np.asarray(X)
        n_samples, n_features = X.shape
        X_scaled = np.empty_like(X, dtype=float)
        for j in range(n_features):
            xmin, xmax = self.X_min_[j], self.X_max_[j]
            if xmax == xmin:
                X_scaled[:, j] = 0
            else:
                X_scaled[:, j] = 2 * (X[:, j] - 0.5)  - 1
        leg_vals = [legendre.legvander(X_scaled[:, j], self.degree) 
                    for j in range(n_features)]

        features = []
        for comb in self.combinations_:
            prod = np.ones(n_samples)
            for j, d in enumerate(comb):
                prod *= leg_vals[j][:, d]
            features.append(prod.reshape(-1, 1))
        return np.hstack(features)


seasons     = ["summer"]
input_names = ["ZR_scale_factor", "ROOF_WIDTH_scale_factor", "ROAD_WIDTH_scale_factor"]
max_degree  = 3
output_base = "/project2/zhan248_1326/hhao4018/UQ_analysis/PCE_urban_nonurban_summaries_v4"

for season in seasons:
    summary_csv = f"/project2/zhan248_1326/hhao4018/UQ_analysis/PCE_urban_nonurban_average_v4/{season}_model_summary.csv"
    input_csv   = f"day_night_citymask_shadow_vars_summer.csv"
    if not os.path.exists(input_csv):
        raise FileNotFoundError(f"Input file not found: {input_csv}")

    out_dir = os.path.join(output_base, season)
    os.makedirs(out_dir, exist_ok=True)

    df = pd.read_csv(summary_csv)
    X_df = pd.read_csv(input_csv)
    merged = pd.merge(df, X_df, on="Model")
    
    for reg in ["city", "non-city"]:
        col = f"SNET_URB_night_{reg}"
        if col in merged.columns:
            merged[col] = 0
        
    for dn in ["day", "night"]:
        for reg in ["city", "non-city"]:
            c1 = f"SNET_URB_{dn}_{reg}"
            c2 = f"LNET_URB_{dn}_{reg}"
            new = f"NET_URB_{dn}_{reg}"
            if c1 in merged.columns and c2 in merged.columns:
                merged[new] = merged[c1] + merged[c2]
    
    X = merged[input_names].to_numpy()

    var_cols = [c for c in merged.columns if c not in ["Model"] + input_names]

    stats = {}
    for var in var_cols:
        y = merged[var].to_numpy()
        mask = ~np.isnan(X).any(axis=1) & ~np.isnan(y)
        X_valid, y_valid = X[mask], y[mask]

        # Cross-validate to pick degree
        mean_scores, std_scores = [], []
        for deg in range(1, max_degree + 1):

            pipeline = Pipeline([
                ("imputer", SimpleImputer(strategy="mean")),
                ("poly", OrthogonalPolynomialFeatures(degree=deg)),
                ("reg", LinearRegression())
            ])
            scores = cross_val_score(
                pipeline, X_valid, y_valid, cv=5, scoring="r2"
            )
            mean_scores.append(scores.mean())
            std_scores.append(scores.std())

        optimal_deg = int(np.argmax(mean_scores) + 1)
        cv_r2       = mean_scores[optimal_deg - 1]

        pipeline = Pipeline([
            ("imputer", SimpleImputer(strategy="mean")),
            ("poly", OrthogonalPolynomialFeatures(degree=optimal_deg)),
            ("reg", LinearRegression())
        ])
        pipeline.fit(X_valid, y_valid)
        reg    = pipeline.named_steps["reg"]
        coeffs = reg.coef_
        poly   = pipeline.named_steps["poly"]
        predicted_mean = reg.intercept_
        
        norms = []
        for comb in poly.combinations_:
            norm = 1.0
            for deg in comb:
                if deg > 0:
                    norm *= 2.0/(2*deg + 1)
            norms.append(norm)
        norms = np.array(norms)

        pce_var = np.sum(coeffs[1:]**2 * norms[1:])
        
        predicted_std = np.sqrt(pce_var/2)

        plt.figure()
        plt.errorbar(
            range(1, max_degree + 1),
            mean_scores, yerr=std_scores, fmt="-o"
        )
        plt.xlabel("Polynomial Degree")
        plt.ylabel("R²")
        plt.title(f"{var} CV R² ({season})")
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, f"CV_{var}.png"))
        plt.close()

        X_imp = SimpleImputer(strategy="mean").fit_transform(X_valid)
        X_scl = StandardScaler().fit_transform(X_imp)
        X_poly = OrthogonalPolynomialFeatures(degree=optimal_deg).fit_transform(X_scl)
        preds = reg.predict(X_poly)
        residuals = y_valid - preds
        plt.figure()
        plt.scatter(preds, residuals, s=10, alpha=0.7)
        plt.axhline(0, color="red", linestyle="--")
        plt.xlabel("Predicted")
        plt.ylabel("Residual")
        plt.title(f"Residuals {var} ({season})")
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, f"Residuals_{var}.png"))
        plt.close()

        raw = coeffs[1:]**2 * norms[1:]/2
        total_var = raw.sum()

        label_map = {0: "H", 1: "W", 2: "R"}

        sobol_idx = {}
        for i in range(3):
            mask = [
                (comb[i] > 0) and (sum(comb) == comb[i])
                for comb in poly.combinations_[1:]
            ]
            sobol_idx[label_map[i]] = (raw[mask].sum() / total_var) if total_var>0 else 0.0

        pair_map = {
            (0, 1): "H*W",
            (0, 2): "H*R",
            (1, 2): "W*R"
        }
        for (i, j), lab in pair_map.items():
            mask = [
                (comb[i] > 0) and (comb[j] > 0) and (sum(comb) == comb[i] + comb[j])
                for comb in poly.combinations_[1:]
            ]
            sobol_idx[lab] = (raw[mask].sum() / total_var) if total_var>0 else 0.0

        labels = ["H", "W", "R", "H*W", "H*R", "W*R"]
        values = [sobol_idx[l] for l in labels]

        plt.figure()
        plt.bar(labels, values)
        plt.xlabel("Parameter")
        plt.ylabel("Sobol Index")
        plt.title(f"Sobol Indices for {var} ({season})")
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, f"Sobol_{var}.png"))
        plt.close()
        stats[var] = {
            "predicted_mean": predicted_mean,
            "predicted_std":  predicted_std,
            "optimal_degree": optimal_deg,
            "cv_r2_mean":     cv_r2,
            **{f"r2_deg_{i+1}": m for i,m in enumerate(mean_scores)},
            **{f"Sobol_{n}":   v for n,v in sobol_idx.items()}
        }

    pd.DataFrame(stats).T.to_csv(
        os.path.join(out_dir, f"PCE_stats_{season}.csv")
    )
    print(f"✅ Completed PCE for season: {season}")
